In [1]:
import json
import os
import shutil
from datetime import datetime, timedelta

import numpy as np
import scipy
import strax
from bson import json_util
from tqdm import tqdm
import lz4.frame as lz4

import helix as hx
from helix import units
import numpy as np
import strax as sx                    
import numba
import pandas as pd
import scipy as sp
import sys
import os
import shutil
from glob import glob
from matplotlib import pyplot as plt

run_id = 'run10' 
duration = 10  # seconds
raw_data_dir = 'toy_data'  # to save the raw toy data
helix_data_dir = 'test_helix_data'  # to save the run metadata
baseline_step = 0  # add a baseline equal to baseline_step*channel_index to each channel
# methods and classes marked with the @export decorator are added to the __all__ namespace to make them importable via
# the star-notation ('from .module_name import *')
export, __all__ = strax.exporter()


@export
def get_pink_psd(trace_length, sampling_dt, noise_std):
    """
    Returns folded PSD corresponding to 1/f pink noise, empirically scaled to make the standard deviation of the noise
    produced from this PSD to be close to the provided noise_std value

    :param trace_length: length of noise traces in samples
    :param sampling_dt: sampling time in ns
    :param noise_std: noise standard deviation
    :return: a tuple of (f, psd), where f is the array of frequencies in Hz, and psd is the array of PSD components
    in A^2/Hz. Can be plotted with plt.loglog(f, psd)
    """
    f = scipy.fft.rfftfreq(trace_length, d=sampling_dt / units.s)
    f[0] = f[1]
    # empirical approximate scaling to match the standard deviation of the resulting noise.
    # This is totally crazy, it's a random expression that scales the PSD in a way that the standard deviation of the
    # noise produced from this PSD somewhat matches the requested noise_std
    # TODO: derive the actual scaling that would properly work and make sense instead of this monster
    scaling = 1 / (1 + (np.log10(trace_length) - 4.5) / 10) ** 2 / 10.3
    psd = scaling*(noise_std**2) / f
    psd[-1] = psd[-1] / 2
    # Could use 0 for the DC component of the PSD, but it messes up the plotting in log scale.
    # The value doesn't matter anyway, we are not generating noise DC components. Using the smallest positive float
    psd[0] = np.nextafter(1, 2)
    f[0] = 0
    return f, psd

def generate_silent_traces(n_traces, psd, sampling_frequency=1.0):
    """
    Function to generate silent traces (zeros) with the same structure as the noise generator.
    :param n_traces: int. Number of traces to generate.
    :param psd: ndarray. Folded power spectral density (not used, but kept for structure consistency).
    :param sampling_frequency: float. Sample frequency in Hz.
    :returns: ndarray. An array of silent (zero) traces.
    """
    trace_length = hx.psd_to_trace_length(len(psd))
    
    # Generate an array of zeros with the appropriate shape
    silent_traces = np.zeros((n_traces, trace_length))
    
    return silent_traces
@export
def load_traces_from_csv(file_path):
    """
    Reads the CSV file containing traces and returns them as a NumPy array.
    :param file_path: str. Path to the CSV file.
    :returns: ndarray. Array of traces with shape (n_traces, 32768).
    """
    traces_df = pd.read_csv(file_path, header=None)
    traces_array = traces_df.to_numpy(dtype=np.float32)
    return traces_array
    
def generate_toy_data(run, duration, directory='toy_data', record_length=hx.DEFAULT_RECORD_LENGTH,
                      sampling_dt=hx.DEFAULT_SAMPLING_DT, template_length=hx.DEFAULT_TEMPLATE_LENGTH,
                      channel_map=hx.DEFAULT_CHANNEL_MAP, noise_std=0, event_rate=1, overwrite=False, helix_data_dir='test_helix_data', baseline_step=0, traces_file='traces.csv'):
    """
    Generates and saves toy data with multiple channels of vacuum and submerged types, physics events consisting of UV
    and QP signals, as well as background lone hits and muon saturated events. CAUTION: it's slow!
    Noise is uncorrelated pink noise, with a correlated 5 kHz feature in all channels. Channels have different baselines

    :param run: run id
    :param duration: run duration in seconds. Caution: the function is slow, don't ask to generate days of data
    :param directory: output directory, where a directory with run_id name will be created
    :param record_length: length of records in each file in time samples
    :param sampling_dt: sampling time in ns
    :param template_length: length of UV and QP templates
    :param channel_map: dictionary of channel types and channel number ranges
    :param noise_std: standard deviation of the noise
    :param event_rate: rate of physics events in Hz
    :param lone_hit_rate: rate of lone background hits in Hz (events with a UV signal in only one channel)
    :param muon_rate: rate of muon events
    :param overwrite: a boolean specifying whether the function should overwrite data, if it already exists. If False,
    a RuntimeError is raise when a directory with the same run id exists
    :param helix_data_dir: a directory to save the run metadata. Should be the same as the helix output directory.
    :param baseline_step: add a baseline to each channel, equal to baseline_step * channel_index
    """

    traces_array = load_traces_from_csv(traces_file)
    run_dir = os.path.join(directory, run)
    if os.path.exists(run_dir):
        if overwrite:
            shutil.rmtree(run_dir)
        else:
            raise RuntimeError(f'Directory {run_dir} already exists.')

    os.makedirs(run_dir)

    record_length_s = record_length * sampling_dt / units.s
    n_records = int(duration / record_length_s)
    channels = hx.Channels(channel_map)
    n_channels = len(channels)
    batch_size = 1  # number of records per batch
    sampling_frequency = 1 / (sampling_dt / units.s)

    _, psd = get_pink_psd(record_length * batch_size, sampling_dt, noise_std)


    # adding different baselines to each channel
    baseline = baseline_step * np.arange(n_channels)    


    n_batches = 1
    # # of baches, if set >1 will have repeat
    #print(f"    n_batches:            {n_batches}")

    batch_length_s = 100
    # 
    #print(f"    batch_length_s:            {batch_length_s}")

    n_events = np.full(n_batches, int(round(event_rate * batch_length_s)), dtype=int)
    print(f"    n_events:            {n_events}")


    for i in tqdm(range(n_batches)):
        #waveform should be generated under each event and has the dimension of (50, 1250000)
        waveform = generate_silent_traces(n_channels, psd, sampling_frequency)

        # generate random event times
        event_times = np.random.randint(0, batch_size * record_length - template_length,
                                        size=n_events[i])

        # adding physics events and muons to the traces
        for ich, ch_type in enumerate(channels.types):
            for j in range(n_events[i]):
                if ich == 0:
                    trace_idx = j % traces_array.shape[0]  # Assign one real trace per event
                    print(f"    trace_idx:            {trace_idx}")
                    selected_trace = traces_array[trace_idx]
                    start_idx = event_times[j] 
                    print(f"    start_idx:            {start_idx}")
                    end_idx = min(start_idx + hx.DEFAULT_TEMPLATE_LENGTH, waveform.shape[1])
                    waveform[ich, event_times[j]:event_times[j] + hx.DEFAULT_TEMPLATE_LENGTH] +=\
                        selected_trace[:]

        # splitting the data into record_length records
        for j in range(batch_size):
            i_record = i * batch_size + j
            if i_record == n_records:
                break
            fn = f'{directory}/{run}/{run}-{i_record:05d}'
            # saving the data as a flattened array
            with open(fn, mode='wb') as f:
                data = np.ascontiguousarray(waveform[:, j * record_length:(j + 1) * record_length], dtype=np.int16)
                f.write(lz4.compress(data))

    # saving run metadata
    if not os.path.exists(helix_data_dir):
        os.makedirs(helix_data_dir)

    metadata_path = os.path.join(helix_data_dir, "%s-metadata.json" % run)
    start = datetime.now().replace(microsecond=0)
    end = start + timedelta(seconds=duration)
    metadata = {'start': start, 'end': end}
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, default=json_util.default)


In [43]:

# remove helix data corresponding to this run_id, if it exists
for path in glob(f'{helix_data_dir}/*'):
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)
    
generate_toy_data(run_id, duration, raw_data_dir, helix_data_dir=helix_data_dir, overwrite=True, baseline_step=baseline_step)


    n_batches:            1
    batch_length_s:            100
    n_events:            [100]


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 12.31it/s]

    trace_idx:            0
    start_idx:            1089019
    trace_idx:            1
    start_idx:            1181111
    trace_idx:            2
    start_idx:            811955
    trace_idx:            3
    start_idx:            198130
    trace_idx:            4
    start_idx:            1182613
    trace_idx:            5
    start_idx:            183995
    trace_idx:            6
    start_idx:            967945
    trace_idx:            7
    start_idx:            1106965
    trace_idx:            8
    start_idx:            62597
    trace_idx:            9
    start_idx:            1110120
    trace_idx:            10
    start_idx:            871270
    trace_idx:            11
    start_idx:            1051858
    trace_idx:            12
    start_idx:            12741
    trace_idx:            13
    start_idx:            797392
    trace_idx:            14
    start_idx:            73004
    trace_idx:            15
    start_idx:            878507
    trace_idx: 

In [ ]:
context = sx.Context(storage=[sx.DataDirectory(helix_data_dir, provide_run_metadata=True), ],
                     register=[hx.MMCRecords,
                               hx.QPTriggers, hx.UVTriggers,
                               hx.Events, hx.NoiseEvents,
                               hx.NoisePSDs, hx.FitResults])    # all the plugins required for getting fit_results

# creating a dictionary of plugins' options that we want to modify. 
config = {'run_metadata_directory': helix_data_dir,      # for the hx.ToyDataRawRecords plugin
          'noise_events_random_seed': 0}  # for the hx.NoiseEvents plugin

context.set_config(config)
events = context.get_array(run_id, 'events')

raw_data = events['channel_data']
raw_data.shape

Removing old incomplete data in test_helix_data/run10-raw_records-igg4zuk5jh_temp
Removing old incomplete data in test_helix_data/run10-qp_triggers-i3vt2yhwmr_temp
Removing old incomplete data in test_helix_data/run10-uv_triggers-xdpckrv34a_temp
Removing old incomplete data in test_helix_data/run10-events-5zxopmcya6_temp


Loading events: |                                             | 0.00 % [00:00<?]

Loading events: |                  | 0.00 % [00:03<?], #78 (0.02 s). 4528.3 MB/s  
(964, 50, 33168)  
Loading fit_results: |                | 0.00 % [03:26<?], #78 (1.16 s). 9.8 kB/s

In [ ]:
fit_results = context.get_array(run_id, 'fit_results')  # hx.FitResults plugin provides this data type

In [ ]:
# Building the channel map and loading the templates
channels = hx.Channels(hx.DEFAULT_CHANNEL_MAP)

template_path = './plugins/event_rqs/default_templates.npy'
templates = np.load(template_path, allow_pickle=True)
uv_template = templates[0]
qp_template = templates[1]


# change this to choose another event if the one below is a bad one by chance
i = 25

plt.figure(figsize=(28,16))
artificial_baselines = np.arange(len(channels)) * 300  # adding artificial baselines to each channel to separate the channels on the plot
plt.plot(events['channel_data'][i].T + artificial_baselines, lw=0.5, alpha=0.8, color='C0')  # plotting data in each channel

event = fit_results[i]

# plotting two-template fits in the vacuum channels
for i_vac, i_ch in enumerate(channels.indices_of_type(hx.ChannelType.VACUUM)):
    # i_vac is the ordinal index of the vacuum channel
    # i_ch is the corrending ordinal index in the array of all channels
    fit = event['vacuum_channel_uv_amplitude'][i_vac] * np.roll(uv_template, event['vacuum_channel_uv_offset'][i_vac]) 
    x = np.arange(len(uv_template)) - hx.DEFAULT_ALLOWED_FIT_SHIFTS[0]  # currently, the FitResults plugin does not fit the entire event. It skips -hx.DEFAULT_ALLOWED_FIT_SHIFTS[0] samples.
    plt.plot(x, fit+artificial_baselines[i_ch], lw=1, alpha=0.8, color='C1')


plt.xlabel('Time (samples)')
plt.ylabel('Current (ADC units)')
plt.title(f'Apply MMC readout on silent traces')
#plt.savefig('raw_traces_on_silent_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [4]:
from helix import units
import numpy as np
import helix as hx
def make_default_templates():
    uv_template = hx.get_analytical_template(length=hx.DEFAULT_OF_LENGTH, sampling_dt=hx.DEFAULT_SAMPLING_DT)
    qp_template = hx.get_analytical_template(2 * units.ms, length=hx.DEFAULT_OF_LENGTH,
                                             sampling_dt=hx.DEFAULT_SAMPLING_DT)
    templates = np.array([uv_template, qp_template])

    np.save('plugins/event_rqs/default_templates.npy', templates)
make_default_templates()